In [1]:
import pandas as pd
gbig_test_df = pd.read_csv("big_vultest_ast.csv")

gbig_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  1170 non-null   object
 1   input        1170 non-null   object
 2   output       1170 non-null   int64 
 3   fol_logic    1170 non-null   object
dtypes: int64(1), object(3)
memory usage: 36.7+ KB


In [6]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './unixcoder_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gbig_test_df = gbig_test_df.rename(columns={"input": "func", "output": "label"})
gbig_test_df = gbig_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
gbig_test_df['model_input'] =gbig_test_df['func'] + SEP +  gbig_test_df['fol_logic']
# Convert to HuggingFace Dataset
gbig_test_ds = Dataset.from_pandas(gbig_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

big_test_tok = gbig_test_ds.map(tokenize_fn, batched=True)
big_test_tok = big_test_tok.rename_column("label", "labels")
big_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=big_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on BigVul Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 1170/1170 [00:01<00:00, 969.94 examples/s]
/tmp/ipykernel_605207/3892405880.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on BigVul Dataset ===
loss        : 0.7859
model_preparation_time: 0.0033
accuracy    : 0.4752
precision   : 0.4707
recall      : 0.3983
specificity : 0.5521
fpr         : 0.4479
f1          : 0.4315
mcc         : -0.0502
kappa       : -0.0496
mse         : 0.5248
mae         : 0.5248
auc         : 0.4656
runtime     : 7.5196
samples_per_second: 155.5940
steps_per_second: 19.5490


In [7]:
import pandas as pd
gdiv_test_df = pd.read_csv("diverse_test_ast.csv")
gdiv_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_snip  1532 non-null   object
 1   output     1532 non-null   int64 
 2   fol_logic  1532 non-null   object
dtypes: int64(1), object(2)
memory usage: 36.0+ KB


In [8]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir ='./unixcoder_trainFOL_testFUNC_saved'

model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gdiv_test_df = gdiv_test_df.rename(columns={"code_snip": "func", "output": "label"})
gdiv_test_df = gdiv_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
gdiv_test_df['model_input'] =gdiv_test_df['func'] + SEP +  gdiv_test_df['fol_logic']
# Convert to HuggingFace Dataset
gdiv_test_ds = Dataset.from_pandas(gdiv_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

div_test_tok = gdiv_test_ds.map(tokenize_fn, batched=True)
div_test_tok = div_test_tok.rename_column("label", "labels")
div_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=div_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Diverse Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 1532/1532 [00:01<00:00, 814.11 examples/s]
/tmp/ipykernel_605207/3980717992.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on Diverse Dataset ===
loss        : 0.7626
model_preparation_time: 0.0017
accuracy    : 0.5117
precision   : 0.5115
recall      : 0.5235
specificity : 0.5000
fpr         : 0.5000
f1          : 0.5174
mcc         : 0.0235
kappa       : 0.0235
mse         : 0.4883
mae         : 0.4883
auc         : 0.5213
runtime     : 9.5518
samples_per_second: 160.3890
steps_per_second: 20.1010


In [9]:
import pandas as pd

# Load the CSV file
dev_test_df = pd.read_csv("devign_test_ast.csv")

# Drop the 'func' column
dev_test_df = dev_test_df.drop(columns=['func'])


# Convert the 'target' column to integer (True → 1, False → 0)
dev_test_df['label'] = dev_test_df['label'].astype(int)
# Check the result
dev_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   func_cleaned  2807 non-null   object
 1   project       2807 non-null   object
 2   label         2807 non-null   int64 
 3   fol_logic     2807 non-null   object
dtypes: int64(1), object(3)
memory usage: 87.8+ KB


In [10]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)


saved_dir ='./unixcoder_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)





dev_test_df = dev_test_df.rename(columns={"func_cleaned": "func", "label": "label"})
dev_test_df = dev_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
dev_test_ds = Dataset.from_pandas(dev_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

devv_test_tok = dev_test_ds.map(tokenize_fn, batched=True)
devv_test_tok = devv_test_tok.rename_column("label", "labels")
devv_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=devv_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Devign Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 2807/2807 [00:11<00:00, 236.82 examples/s]
/tmp/ipykernel_605207/2738335746.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on Devign Dataset ===
loss        : 0.6333
model_preparation_time: 0.0016
accuracy    : 0.6352
precision   : 0.5861
recall      : 0.5350
specificity : 0.7117
fpr         : 0.2883
f1          : 0.5594
mcc         : 0.2500
kappa       : 0.2492
mse         : 0.3648
mae         : 0.3648
auc         : 0.6907
runtime     : 17.6863
samples_per_second: 158.7110
steps_per_second: 19.8460


In [11]:
import pandas as pd
juliet_test_df = pd.read_csv("djuliet_test_ast.csv")
juliet_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3152 entries, 0 to 3151
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_snip  3152 non-null   object
 1   output     3152 non-null   int64 
 2   fol_logic  3152 non-null   object
dtypes: int64(1), object(2)
memory usage: 74.0+ KB


In [12]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir ='./unixcoder_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
juliet_test_df = juliet_test_df.rename(columns={"code_snip": "func", "output": "label"})
juliet_test_df = juliet_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
juliet_test_df['model_input'] =juliet_test_df['func'] + SEP +  juliet_test_df['fol_logic']
# Convert to HuggingFace Dataset
juliet_test_ds = Dataset.from_pandas(juliet_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

juliet_test_tok = juliet_test_ds.map(tokenize_fn, batched=True)
juliet_test_tok = juliet_test_tok.rename_column("label", "labels")
juliet_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=juliet_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on juliet Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 3152/3152 [00:02<00:00, 1266.08 examples/s]
/tmp/ipykernel_605207/3099362496.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on juliet Dataset ===
loss        : 0.6689
model_preparation_time: 0.0018
accuracy    : 0.5435
precision   : 0.5710
recall      : 0.3496
specificity : 0.7373
fpr         : 0.2627
f1          : 0.4337
mcc         : 0.0943
kappa       : 0.0869
mse         : 0.4565
mae         : 0.4565
auc         : 0.6229
runtime     : 19.8056
samples_per_second: 159.1470
steps_per_second: 19.8930


In [13]:
import pandas as pd
reveal_test_df = pd.read_csv("reveal_vultest_ast.csv")
reveal_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2028 entries, 0 to 2027
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2028 non-null   object
 1   input        2028 non-null   object
 2   output       2028 non-null   int64 
 3   fol_logic    2028 non-null   object
dtypes: int64(1), object(3)
memory usage: 63.5+ KB


In [14]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir ='./unixcoder_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
reveal_test_df = reveal_test_df.rename(columns={"input": "func", "output": "label"})
reveal_test_df = reveal_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
reveal_test_df['model_input'] =reveal_test_df['func'] + SEP +  reveal_test_df['fol_logic']
# Convert to HuggingFace Dataset
reveal_test_ds = Dataset.from_pandas(reveal_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

reveal_test_tok = reveal_test_ds.map(tokenize_fn, batched=True)
reveal_test_tok = reveal_test_tok.rename_column("label", "labels")
reveal_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=reveal_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on RVl Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 2028/2028 [00:02<00:00, 849.25 examples/s]
/tmp/ipykernel_605207/2681239086.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on RVl Dataset ===
loss        : 0.8276
model_preparation_time: 0.0021
accuracy    : 0.4911
precision   : 0.4828
recall      : 0.2485
specificity : 0.7337
fpr         : 0.2663
f1          : 0.3281
mcc         : -0.0203
kappa       : -0.0178
mse         : 0.5089
mae         : 0.5089
auc         : 0.4842
runtime     : 12.7569
samples_per_second: 158.9730
steps_per_second: 19.9110


In [16]:
import pandas as pd
cv_test_df = pd.read_csv("cevfix_test_ast.csv")
cv_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  4216 non-null   object
 1   input        4216 non-null   object
 2   output       4216 non-null   int64 
 3   idx          4216 non-null   int64 
 4   fol_logic    4216 non-null   object
dtypes: int64(2), object(3)
memory usage: 164.8+ KB


In [17]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir ='./unixcoder_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
cv_test_df = cv_test_df.rename(columns={"input": "func", "output": "label"})
cv_test_df = cv_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
cv_test_df['model_input'] =cv_test_df['func'] + SEP +  cv_test_df['fol_logic']
# Convert to HuggingFace Dataset
cv_test_ds = Dataset.from_pandas(cv_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

cv_test_tok = cv_test_ds.map(tokenize_fn, batched=True)
cv_test_tok = cv_test_tok.rename_column("label", "labels")
cv_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=cv_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on RVl Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 4216/4216 [00:07<00:00, 600.19 examples/s]
/tmp/ipykernel_605207/2479635731.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on RVl Dataset ===
loss        : 0.7642
model_preparation_time: 0.0018
accuracy    : 0.5007
precision   : 0.5007
recall      : 0.5024
specificity : 0.4991
fpr         : 0.5009
f1          : 0.5015
mcc         : 0.0014
kappa       : 0.0014
mse         : 0.4993
mae         : 0.4993
auc         : 0.5077
runtime     : 26.1604
samples_per_second: 161.1590
steps_per_second: 20.1450


In [20]:
import pandas as pd
CV_test_df = pd.read_csv("mix_vultest_ast.csv")
CV_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2864 non-null   object
 1   input        2864 non-null   object
 2   output       2864 non-null   int64 
 3   fol_logic    2864 non-null   object
dtypes: int64(1), object(3)
memory usage: 89.6+ KB


In [21]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir ='./unixcoder_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
CV_test_df = CV_test_df.rename(columns={"input": "func", "output": "label"})
CV_test_df = CV_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
CV_test_df['model_input'] =CV_test_df['func'] + SEP +  CV_test_df['fol_logic']
# Convert to HuggingFace Dataset
CV_test_ds = Dataset.from_pandas(CV_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

CV_test_tok = CV_test_ds.map(tokenize_fn, batched=True)
CV_test_tok = CV_test_tok.rename_column("label", "labels")
CV_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=CV_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on RVl Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 2864/2864 [00:04<00:00, 708.82 examples/s]
/tmp/ipykernel_605207/2397350884.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on RVl Dataset ===
loss        : 0.6998
model_preparation_time: 0.0020
accuracy    : 0.5681
precision   : 0.5685
recall      : 0.5649
specificity : 0.5712
fpr         : 0.4288
f1          : 0.5667
mcc         : 0.1362
kappa       : 0.1362
mse         : 0.4319
mae         : 0.4319
auc         : 0.5986
runtime     : 17.9268
samples_per_second: 159.7610
steps_per_second: 19.9700
